In [1]:
!pip install -q pyomo

In [2]:
from pyomo.environ import *
import numpy as np
import pandas as pd
!apt-get install -y -qq glpk-utils
solver = SolverFactory('glpk', executable='/usr/bin/glpsol')

In [3]:
matrix_A = pd.read_csv("/content/incidence_matrix.csv")

In [4]:
matrix_A.head(11)

,Unnamed: 0,1,2,3,4,5,6,7,8
0,1,1,0,0,1,0,0,1,0
1,2,0,1,0,0,1,0,0,0
2,3,0,0,1,0,0,1,0,1
3,4,1,0,0,1,0,0,0,0
4,5,0,0,1,1,0,1,0,0
5,6,0,1,0,1,0,1,0,0
6,7,0,0,1,0,1,0,0,0
7,8,0,0,1,0,1,0,1,0
8,9,0,1,0,1,0,1,0,0
9,10,1,0,0,0,0,0,0,1


In [5]:
data = matrix_A.iloc[:,1:]
print(data)

   1  2  3  4  5  6  7  8
0  1  0  0  1  0  0  1  0
1  0  1  0  0  1  0  0  0
2  0  0  1  0  0  1  0  1
3  1  0  0  1  0  0  0  0
4  0  0  1  1  0  1  0  0
5  0  1  0  1  0  1  0  0
6  0  0  1  0  1  0  0  0
7  0  0  1  0  1  0  1  0
8  0  1  0  1  0  1  0  0
9  1  0  0  0  0  0  0  1


In [6]:
#cost
C = [5,4,4,9,7,8,3,3]

In [7]:
model = ConcreteModel()

In [8]:
col = range(len(C)) #col  indices
print(col)

range(0, 8)


In [9]:
model.y = Var(col , domain = Binary)

In [10]:
model.obj = Objective(expr = summation(C,model.y)  , sense = minimize)

In [11]:
model.constr = ConstraintList()

In [12]:
rhs = [1,1,1,1,1,1,1,1,1,1] #set covering rhs

In [13]:
for i in range(10):
  model.constr.add(expr = summation(data.iloc[i,:] , model.y) >= rhs[i])

In [14]:
model.pprint()

2 Set Declarations
    constr_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    y_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    8 : {0, 1, 2, 3, 4, 5, 6, 7}

1 Var Declarations
    y : Size=8, Index=y_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary

1 Objective Declarations
    obj : Size=1, Inde

In [15]:
result = solver.solve(model)

In [16]:
print('Solver Termination Condition : ',result.solver.termination_condition)
print('Solver Status : ' , result.solver.status)

Solver Termination Condition :  optimal
Solver Status :  ok


In [17]:
print('optimal value of cost : ' , model.obj())
print("\nvariables")
for i in col :
  print('value of y [',i+1,'] :' , model.y[i].value)

optimal value of cost :  13.0

variables
value of y [ 1 ] : 1.0
value of y [ 2 ] : 1.0
value of y [ 3 ] : 1.0
value of y [ 4 ] : 0.0
value of y [ 5 ] : 0.0
value of y [ 6 ] : 0.0
value of y [ 7 ] : 0.0
value of y [ 8 ] : 0.0


In [18]:
print("feasible flight sequence is :")
for i in col :
  if (model.y[i] != 0 ) :
    print('y[',i+1,']')


feasible flight sequence is :
y[ 1 ]
y[ 2 ]
y[ 3 ]
